In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict
import math

In [2]:
#Hadamard matrices

H = {}
H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])


f = open("had64.txt")
H64 = []
for i in range(64):
    H64.append([1.0 if i=='+' else -1.0 for i in list(f.readline())[:-1]])
    
H[64] = H64

In [3]:
#Generating boolean functions and from them, their walsh spectra to feed as inputs to the neural network

#k = number of variables in the boolean functions, n = number of inputs to train the model on
def generate_input_data(k, n, scheme='regular'):
    two_pow_k, data = pow(2, k), []
    if(scheme=='regular'):
        if(n<two_pow_k):
            print("Requested number of inputs less than 2^k. Please request a higher number")
            return data
        while(True):
            if(two_pow_k<=16):
                perm = torch.randperm(pow(2, two_pow_k))
                data = [[0.0]*(two_pow_k-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in perm[:n]]
            else:
                data = [[0.0 if random.random()>0.5 else 1.0 for i in range(two_pow_k)] for i in range(n)]
            rank = np.linalg.matrix_rank(data)
            if(rank<two_pow_k):
                print("Rank (", rank, ") not large enough, generating data again")
            else:
                data = torch.matmul(torch.tensor(data), H[two_pow_k])
                print("Data generated.")
                break
    elif(scheme=='one-hot'):
        data = [[0.0]*i + [1.0] + [0.0]*(two_pow_k-i-1) for i in range(two_pow_k)]
        data = torch.matmul(torch.tensor(data), H[two_pow_k])
        print("Data generated")
    return data

k = 3
n = pow(2, pow(2, k))
walsh_spectra = generate_input_data(k, n)

Data generated.


In [77]:
#Calculating the correlation immunity of functions given their walsh spectra

def correlation_immunity(walsh_spectra):
    ci = []
    n, two_pow_k = walsh_spectra.size()[0], walsh_spectra.size()[1]
    k = int(math.log2(two_pow_k))
    no_ones = []
    for i in range(two_pow_k):
        no_ones.append(sum([int(dig) for dig in bin(i)[2:]]))
    for spectrum in walsh_spectra:
        m_ci = [1]*(k+1)
        for i in range(two_pow_k):
            if(spectrum[i]!=0):
                m_ci[no_ones[i]] = 0
        m = 1
        while(m<k+1 and m_ci[m]==1):
            m+=1
        m -= 1
        # Let ci item = [x0, x1, ..., xm]. x0 is 1 if it is balanced, xi is 1 if it is i-correlation-immune
        ci.append(([1.0] if spectrum[0]==two_pow_k//2 else [0.0])+(m)*[1.0]+(k-m)*[0.0])
    return ci

ci = torch.tensor(correlation_immunity(walsh_spectra))
#temp
ci = ci[:, 0]

In [78]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        two_pow_k = pow(2,k)
        self.linear_stack = nn.Sequential(
            nn.Linear(two_pow_k, two_pow_k),
            nn.ReLU(),
            nn.Linear(two_pow_k, two_pow_k),
            nn.ReLU(),
            nn.Linear(two_pow_k, two_pow_k),
            nn.ReLU(),
            nn.Linear(two_pow_k, two_pow_k),
            nn.ReLU(),
#             nn.Linear(two_pow_k, k+1),
            nn.Linear(two_pow_k, 1),
        )

    def forward(self, x):
        immunity = self.linear_stack(x)
#         print(self.linear_stack(x))
#         immunity = torch.tensor([[1.0 if imm<=0 else 0.0 for imm in inp] for inp in self.linear_stack(x)])
        return immunity

In [79]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

model = NeuralNetwork()

Using cpu device


In [80]:
class TrainModel():

    def __init__(self, model, device, learningRate, inputDim, epochs):

        self.device = device
        self.net = model.to(self.device)
        self.optimizer = optim.SGD(self.net.parameters(), lr=learningRate)
        self.inputDim = inputDim
        self.epochs = epochs

    def train(self,):

        for epoch in range(self.epochs):

            self.optimizer.zero_grad()
            input = walsh_spectra.to(self.device)
            output = self.net(input)
            loss = F.mse_loss(torch.tensor(ci), output)
            loss.backward()
            self.optimizer.step()      

    def test(self,n):

        test_samples = walsh_spectra.to(self.device)[:n]
        preds = self.net(test_samples)
        loss = F.mse_loss(torch.tensor(ci[:n]), preds)
        print("loss = ", loss)
        print(ci[:n])
        print(preds)


In [81]:
learningRate = 0.001
epochs = 1000
two_pow_k = pow(2, k)

# Additional Initializer
model =  NeuralNetwork()
trainer = TrainModel(model, device, learningRate, two_pow_k, epochs)
trainer.train()
trainer.test(10)


/tmp/ipykernel_1768032/3145797889.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.mse_loss(torch.tensor(ci), output)
/tmp/ipykernel_1768032/3145797889.py:18: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(torch.tensor(ci), output)


loss =  tensor(0.3089, grad_fn=<MseLossBackward0>)
tensor([0., 1., 0., 0., 0., 1., 1., 1., 0., 1.])
tensor([[0.2549],
        [0.2860],
        [0.2501],
        [0.2738],
        [0.2221],
        [0.2824],
        [0.2403],
        [0.2654],
        [0.2797],
        [0.2275]], grad_fn=<AddmmBackward0>)


/tmp/ipykernel_1768032/3145797889.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.mse_loss(torch.tensor(ci[:n]), preds)
/tmp/ipykernel_1768032/3145797889.py:26: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(torch.tensor(ci[:n]), preds)


In [56]:
x = torch.tensor([[ 3.3839e-02, -1.1122e-01, -4.7419e-01],
        [-8.8419e-03, -2.9123e-01, -3.9918e-01],
        [ 4.7317e-02, -2.3600e-01, -3.7276e-01],
        [ 3.7765e-01, -2.1951e-02, -6.0302e-01],
        [ 1.2820e-01,  6.2765e-02, -5.1973e-01]])

# for i, inp in enumerate(x):
for inp in x:
    for imm in inp:
        if(imm<=0):
            print(imm)
# immunity = torch.tensor([torch.tensor([1.0 if imm<=0 else 0.0 for imm in enumerate(inp)]) for inp in enumerate(x)])
immunity = torch.tensor([[1.0 if imm<=0 else 0.0 for imm in inp] for inp in x])
print(immunity)

for x in immunity:
    print(x)


# print(len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]))
# print(walsh_spectra[240])
# print(correlation_immunity(walsh_spectra)[240])

tensor(-0.1112)
tensor(-0.4742)
tensor(-0.0088)
tensor(-0.2912)
tensor(-0.3992)
tensor(-0.2360)
tensor(-0.3728)
tensor(-0.0220)
tensor(-0.6030)
tensor(-0.5197)
tensor([[0., 1., 1.],
        [1., 1., 1.],
        [0., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])
tensor([0., 1., 1.])
tensor([1., 1., 1.])
tensor([0., 1., 1.])
tensor([0., 1., 1.])
tensor([0., 0., 1.])
